In [5]:
import sys
print(sys.executable)
print(sys.version)


/opt/homebrew/opt/python@3.11/bin/python3.11
3.11.13 (main, Jun  3 2025, 18:38:25) [Clang 17.0.0 (clang-1700.0.13.3)]


In [10]:
!/opt/homebrew/opt/python@3.11/bin/python3.11 --version
!/opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install mediapipe opencv-python


Python 3.11.13
  Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 48.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 53.8 MB/s eta 0:00:00a 0:00:01
Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl (37.3 MB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 MB 61.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.4/671.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 76.2 MB/s eta 0:00:00a 0:00

In [6]:
!/opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip setuptools wheel
!/opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install pygame --use-pep517
!/opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install box2d-py --use-pep517


  Using cached pygame-2.6.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 58.7 MB/s eta 0:00:00a 0:00:01


In [7]:
import pygame
import Box2D
print("pygame and Box2D are installed successfully!")


pygame and Box2D are installed successfully!


/opt/homebrew/lib/python3.11/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [ ]:
# the game simulation
import cv2
import mediapipe as mp
import gym
import numpy as np
import time

# Initialize MediaPipe and Gym
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
# Increase max_episode_steps to lengthen max episode duration (default 1000 for LunarLander-v2)
env = gym.make("LunarLander-v2", render_mode="human", max_episode_steps=1500)  
obs = env.reset()

def get_finger_states(hand_landmarks):
    lm = hand_landmarks.landmark
    fingers = []

    palm_facing = lm[0].z < lm[5].z

    # Thumb
    thumb_up = lm[4].x < lm[3].x if palm_facing else lm[4].x > lm[3].x
    fingers.append(thumb_up)

    # Index to pinky
    for tip, pip in [(8, 6), (12, 10), (16, 14), (20, 18)]:
        up = lm[tip].y < lm[pip].y if palm_facing else lm[tip].y > lm[pip].y
        fingers.append(up)

    return fingers

def gesture_to_action(fingers):
    thumb, index, middle, _, _ = fingers
    if index and middle:
        return 2
    elif index and not middle and not thumb:
        return 1
    elif thumb and not index and not middle:
        return 3
    else:
        return 0

cap = cv2.VideoCapture(0)

with mp_hands.Hands(max_num_hands=1,
                    min_detection_confidence=0.7,
                    min_tracking_confidence=0.7) as hands:

    done = False
    total_reward = 0
    while not done:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)

        action = 0
        if results.multi_hand_landmarks:
            for hand in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)
                fingers = get_finger_states(hand)
                action = gesture_to_action(fingers)
                break

        # Frame skip: repeat same action for N steps to slow control updates and extend time
        frame_skip = 4
        for _ in range(frame_skip):
            obs, reward, done, truncated, info = env.step(action)
            total_reward += reward
            if done:
                break

        cv2.putText(frame, f'Action: {action}', (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.imshow("Hand Gesture Control", frame)

        time.sleep(0.1)  # Slightly bigger delay (100ms)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    env.close()
    cap.release()
    cv2.destroyAllWindows()

    if total_reward >= 200:
        print(f"You won! Total reward: {total_reward:.2f}")
    else:
        print(f"Game over. Total reward: {total_reward:.2f}")


I0000 00:00:1749431352.339507 19292162 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1749431352.359611 19299647 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749431352.377417 19299653 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Game over. Total reward: -298.16
